In [1]:
from google.cloud import vision
from google.cloud import storage
from google.protobuf import json_format
from google.cloud import translate
from google.cloud import bigquery
from google.cloud import datastore
import logging

import re
import time
import pandas as pd
import numpy as np

#!sudo pip3 install scispacy
import scispacy
import spacy
from spacy import displacy
#https://github.com/explosion/spacy-models/releases/download/en_core_sci_sm-2.2.0/en_core_sci_sm-2.2.0.tar.gz
#https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_lg-0.2.4.tar.gz
import en_core_sci_sm, en_core_sci_lg #, en_ner_bionlp13cg_md
from scispacy.umls_linking import UmlsEntityLinker
from scispacy.abbreviation import AbbreviationDetector

References:
- ScispaCy: Fast and Robust Models for Biomedical Natural Language Processing

In [2]:
project_id = "pm-preparation"
location = "us-central1"
storage_client = storage.Client()
vision_client = vision.ImageAnnotatorClient()
translate_client = translate.TranslationServiceClient()
datastore_client = datastore.Client()
bq_client = bigquery.Client()

In [3]:
def async_detect_document(vision_client, gcs_source_uri, gcs_destination_uri):
    """OCR with PDF/TIFF as source files on GCS"""
    doc_title = gcs_source_uri.split('/')[-1].split('.pdf')[0]
    
    # Supported mime_types are: 'application/pdf' and 'image/tiff'
    mime_type = 'application/pdf'

    # How many pages should be grouped into each json output file.
    batch_size= 20

    feature = vision.types.Feature(
        type=vision.enums.Feature.Type.DOCUMENT_TEXT_DETECTION)

    gcs_source = vision.types.GcsSource(uri=gcs_source_uri)
    input_config = vision.types.InputConfig(
        gcs_source=gcs_source, mime_type=mime_type)

    gcs_destination = vision.types.GcsDestination(uri=gcs_destination_uri)
    output_config = vision.types.OutputConfig(
        gcs_destination=gcs_destination, batch_size=batch_size)

    async_request = vision.types.AsyncAnnotateFileRequest(
        features=[feature], input_config=input_config,
        output_config=output_config)

    operation = vision_client.async_batch_annotate_files(
        requests=[async_request])

    #print('Waiting for the operation to finish.')
    operation.result(timeout=180)
    print('Text extraction from document {} is completed.'.format(doc_title))


In [4]:
# Once the request has completed and the output has been
# written to GCS, we can list all the output files.
def read_json_result(json_gcs_path, doc_title):
    gcs_destination_prefix = 'json/' + '{}-'.format(doc_title)
    
    # List objects with the given prefix.
    blob_list = list(storage_client.list_blobs(bucket_or_name=bucket_name,
                                               prefix=gcs_destination_prefix))
    all_text = ''
    for blob in blob_list:

        json_string = blob.download_as_string()
        response = json_format.Parse(
            json_string, vision.types.AnnotateFileResponse())

        # The actual response for the first page of the input file.
        
        for response in response.responses:   
        #first_page_response = response.responses[0]
            text_response = response.full_text_annotation.text
            all_text += text_response
            all_text += ' '

    print("Parsed json doc: {}".format(doc_title))
    return all_text

In [5]:
def upload_blob(txt_content, destination_blob_name):
    """Uploads a file to the bucket."""
    destination_blob_name = destination_blob_name.split('gs://{}/'.format(bucket_name))[-1]
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_string(txt_content)

    print("Text uploaded to {}".format(destination_blob_name))

In [6]:
def batch_translate_text(
    input_uri="gs://YOUR_BUCKET_ID/path/to/your/file.txt",
    output_uri="gs://YOUR_BUCKET_ID/path/to/save/results/"):
    """Translates a batch of texts on GCS and stores the result in a GCS location."""

    # Supported file types: https://cloud.google.com/translate/docs/supported-formats
    gcs_source = {"input_uri": input_uri}

    input_configs_element = {
        "gcs_source": gcs_source,
        "mime_type": "text/plain"  # Can be "text/plain" or "text/html".
    }
    gcs_destination = {"output_uri_prefix": output_uri}
    output_config = {"gcs_destination": gcs_destination}
    parent = translate_client.location_path(project_id, location)

    # Supported language codes: https://cloud.google.com/translate/docs/language
    operation = translate_client.batch_translate_text(
        parent=parent,
        source_language_code="it",
        target_language_codes=["en"],  # Up to 10 language codes here.
        input_configs=[input_configs_element],
        output_config=output_config)

    response = operation.result(180)

In [7]:
def removePunctuation(string): 
  
    # punctuation marks 
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  
    # traverse the given string and if any punctuation 
    # marks occur replace it with null 
    for x in string.lower(): 
        if x in punctuations: 
            string = string.replace(x, "") 
  
    # Print string without punctuation 
    return string

**This step will take 1hr and 20 min approx**

In [8]:
customize_stop_words = [
    'uoc', 'diagnostic', 'interventional', 'radiology', 'madonna', 'delle', 'grazie', 'hospital',
    'Borgheresi', 'Agostini', 'Ottaviani', 'Floridi', 'Giovagnoni', 'di', 'specialization',
    'Polytechnic', 'University', 'marche', 'ANCONA', 'Italy', 'Azienda', 'Ospedali', 
    'Riuniti', 'Yorrette', 'Matera', 'Michele', 'Nardella', 'Gerardo', 'Costanzo',
    'Claudia', 'Lopez', 'st', 'a.', 'a', 'of', 's', 'cien', 'ze', 'diolog', 'ic', 'he',
    'â', '€','s','b','case','Cuoladi','l','c','ra','bergamo','patelli','est','asst',
    'dr','Dianluigi', 'Svizzero','i','riccardo','Alessandro','Spinazzola','angelo',
    'maggiore', 'p' ,'r' ,'t', 'm', 'en', 't', 'o', 'd', 'e', 'n', 'd', 'o', 'g', 'h', 'u'
]

In [15]:
# Process documents

bucket_name = 'covid19-public-dataset-aketari'
gcs_source_prefix = 'pdf'
lst_pdf_blobs = storage_client.list_blobs(bucket_or_name=bucket_name, 
                                      prefix='pdf')

lst_json_blobs = storage_client.list_blobs(bucket_or_name=bucket_name, 
                                      prefix='json')

overall_start_time = time.time()
for blob in lst_pdf_blobs:
    doc_title = blob.name.split('/')[-1].split('.pdf')[0]
    #files_metadata[doc_title] = {}

    # Generate all paths
    gcs_source_path = 'gs://' + bucket_name +'/' + blob.name

    #start_time = time.time()
    # OCR pdf documents
    async_detect_document(vision_client, 
                        gcs_source_path,
                        json_gcs_dest_path)
print ('OCR done.')
    
for blob in lst_json_blobs:
    doc_title = blob.name.split('/')[-1].split('-')[0]
    
    json_gcs_dest_path = 'gs://' + bucket_name + '/{}'.format(blob.name)
    txt_gcs_dest_path = 'gs://' + bucket_name + '/raw_txt/' + doc_title + '.txt'
    eng_txt_gcs_dest_path = 'gs://' + bucket_name + '/eng_txt/{}/'.format(doc_title)
    processed_eng_gcs_dest_path = 'gs://' + bucket_name + '/curated_eng_txt/' + doc_title + '.txt'
    
    # Parse json
    all_text = read_json_result(json_gcs_dest_path, doc_title)
    #files_metadata[doc_title]['text'] = all_text

    # Upload raw text to GCS
    upload_blob(all_text, txt_gcs_dest_path)

    # Translate raw text to english
    batch_translate_text(input_uri = txt_gcs_dest_path,
                        output_uri = eng_txt_gcs_dest_path)
    
    # Process eng raw text
    blob_prefix = 'eng_txt/{}/{}_raw_txt_{}_en_translations.txt'.format(doc_title,
                                                                         bucket_name,
                                                                         doc_title)
    
    eng_blob = storage_client.get_bucket(bucket_name).get_blob(blob_prefix)
    eng_raw_string = eng_blob.download_as_string().decode('utf-8')
    
    # lowercase
    #sample_text = eng_raw_string.lower()

    # Remove dates
    # 1 or 2 digit number followed by back slash followed by 1 or 2 digit number ...
    pattern_dates = '(\d{1,2})/(\d{1,2})/(\d{4})'
    pattern_fig = 'Figure (\d{1,2})' 
    pattern_image = '^Image .$'
    replace = ''

    eng_raw_string = re.sub(pattern_dates, replace, eng_raw_string) 
    eng_raw_string = re.sub(pattern_fig, replace, eng_raw_string)
    eng_raw_string = re.sub(pattern_image, replace, eng_raw_string) 
    
    # remove punctuation and special characters
    eng_raw_string = re.sub('[^A-Za-z0-9]+', ' ', eng_raw_string)

    # Remove custom stop words
    tokens = [token for token in eng_raw_string.split()if token not in customize_stop_words]

    refined_doc = ''
    for word in tokens:
        refined_doc += ' {}'.format(word)
    
    # Upload raw text to GCS
    upload_blob(refined_doc, processed_eng_gcs_dest_path)

    #print('refinement completed')
    print('{} processing is done.'.format(doc_title))


Parsed json doc: case1
Text uploaded to raw_txt/case1.txt
Text uploaded to curated_eng_txt/case1.txt
case1 processing is done.
Parsed json doc: case10
Text uploaded to raw_txt/case10.txt
Text uploaded to curated_eng_txt/case10.txt
case10 processing is done.
Parsed json doc: case11
Text uploaded to raw_txt/case11.txt
Text uploaded to curated_eng_txt/case11.txt
case11 processing is done.
Parsed json doc: case12
Text uploaded to raw_txt/case12.txt
Text uploaded to curated_eng_txt/case12.txt
case12 processing is done.
Parsed json doc: case13
Text uploaded to raw_txt/case13.txt
Text uploaded to curated_eng_txt/case13.txt
case13 processing is done.
Parsed json doc: case14
Text uploaded to raw_txt/case14.txt
Text uploaded to curated_eng_txt/case14.txt
case14 processing is done.
Parsed json doc: case15
Text uploaded to raw_txt/case15.txt
Text uploaded to curated_eng_txt/case15.txt
case15 processing is done.
Parsed json doc: case16
Text uploaded to raw_txt/case16.txt
Text uploaded to curated_en

## Upload to bigquery

In [16]:
def bqCreateDataset(dataset_name):
    
    dataset_ref = bq_client.dataset(dataset_name)

    try:
        return bq_client.get_dataset(dataset_ref).dataset_id
    except:
        dataset = bigquery.Dataset(dataset_ref)
        dataset = bq_client.create_dataset(dataset)
        print('Dataset {} created.'.format(dataset.dataset_id))
        return dataset.dataset_id

In [17]:
def bqCreateTable(dataset_id, 
                  table_name,):
    """
    Create main table with all cases and the medical text.
    return:
        table_id
    """
    dataset_ref = bq_client.dataset(dataset_id)

    # Prepares a reference to the table
    table_ref = dataset_ref.table(table_name)

    try:
        return bq_client.get_table(table_ref).table_id
    except:
        schema = [
            bigquery.SchemaField('case', 'STRING', mode='REQUIRED'),
            bigquery.SchemaField('it_raw_txt', 'STRING', mode='REQUIRED'),
            bigquery.SchemaField('eng_raw_txt', 'STRING', mode='REQUIRED'),
            bigquery.SchemaField('eng_txt', 'STRING', mode='REQUIRED', 
                                 description='Output of preprocessing pipeline.')]
        table = bigquery.Table(table_ref, schema=schema)
        table = bq_client.create_table(table)
        print('table {} created.'.format(table.table_id))
        return table.table_id

In [18]:
def exportItems2BQ(dataset_id, table_id, case,
                   it_raw_blob, eng_raw_blob, curated_eng_blob):

    # Prepares a reference to the dataset
    dataset_ref = bq_client.dataset(dataset_id)

    table_ref = dataset_ref.table(table_id)
    table = bq_client.get_table(table_ref)  # API call
    
    # Download text from GCS
    it_raw_txt_string = it_raw_blob.download_as_string().decode('utf-8')
    eng_raw_txt_string = eng_raw_blob.download_as_string().decode('utf-8')
    curated_eng_string = curated_eng_blob.download_as_string().decode('utf-8')
    
    rows_to_insert = [{'case': case,
                      'it_raw_txt': it_raw_txt_string,
                      'eng_raw_txt': eng_raw_txt_string,
                      'eng_txt': curated_eng_string
                     }]
    errors = bq_client.insert_rows(table, rows_to_insert)  # API request
    assert errors == []
    print('{} was added to {} dataset, specifically in {} table.'.format(case,
                                                                        dataset_id,
                                                                        table_id))

In [19]:
bucket_name = 'covid19-public-dataset-aketari'
gcs_source_prefix = 'raw_txt'
dataset_id = bqCreateDataset('covid19')
table_id = bqCreateTable(dataset_id, 'ISMIR_cases')

lst_blobs = storage_client.list_blobs(bucket_or_name=bucket_name, 
                                      prefix=gcs_source_prefix)


for blob in lst_blobs:
    doc_title = blob.name.split('/')[-1].split('.txt')[0]
    
    # download as string
    # it_raw_txt = gs://bucket_name/
    it_raw_blob = storage_client.get_bucket(bucket_name).get_blob('raw_txt/{}.txt'.format(doc_title))
    
    # eng_raw_txt = gs://covid19-aziz/text/[...]covid19-aziz_text_raw_txt_{doc_title}_en_translations.txt
    path_blob_eng_raw = 'eng_txt/{}/{}_raw_txt_{}_en_translations.txt'.format(doc_title,
                                                                              bucket_name,
                                                                              doc_title)
    eng_raw_blob = storage_client.get_bucket(bucket_name).get_blob(path_blob_eng_raw)
    
    # curated_eng_txt = gs://covid19-aziz/text/curated_eng_txt/case1.txt
    curated_eng_blob = storage_client.get_bucket(bucket_name)\
                                    .get_blob('curated_eng_txt/{}.txt'.format(doc_title))
    
    # populate to BQ dataset
    exportItems2BQ(dataset_id, table_id, doc_title,
                   it_raw_blob, eng_raw_blob, curated_eng_blob)


Dataset covid19 created.
table ISMIR_cases created.
case1 was added to covid19 dataset, specifically in ISMIR_cases table.
case10 was added to covid19 dataset, specifically in ISMIR_cases table.
case11 was added to covid19 dataset, specifically in ISMIR_cases table.
case12 was added to covid19 dataset, specifically in ISMIR_cases table.
case13 was added to covid19 dataset, specifically in ISMIR_cases table.
case14 was added to covid19 dataset, specifically in ISMIR_cases table.
case15 was added to covid19 dataset, specifically in ISMIR_cases table.
case16 was added to covid19 dataset, specifically in ISMIR_cases table.
case17 was added to covid19 dataset, specifically in ISMIR_cases table.
case18 was added to covid19 dataset, specifically in ISMIR_cases table.
case19 was added to covid19 dataset, specifically in ISMIR_cases table.
case2 was added to covid19 dataset, specifically in ISMIR_cases table.
case20 was added to covid19 dataset, specifically in ISMIR_cases table.
case21 was add

In [45]:
def returnQueryResults(bq_client, project_id, dataset_id, table_id, case_id):
    """

    Args:
        bq_client:
        project_id:
        dataset_id:
        table_id:
        case_id:

    Returns:

    """

    query = ('SELECT * FROM `{}.{}.{}` WHERE `case`="{}" LIMIT 1'.format(project_id, dataset_id, table_id, case_id))

    try:
        query_job = bq_client.query(query)
        is_exist = len(list(query_job.result())) >= 1
        logging.info('Query case id: {}'.format(case_id) if is_exist \
                         else "Case id: {} does NOT exist".format(case_id))
        print (list(query_job.result()))
    except Exception as e:
        logging.error("Error", e)

In [46]:
returnQueryResults(bq_client, project_id, 'covid19', 'ISMIR_cases', 'case1')

[Row(('case1', 'COVID-19: caso 1\na cura di:\nA. Borgheresi, A. Agostini, L. Ottaviani, C. Floridi, A. Giovagnoni\nDi p a r t i m en t o d i S c i e n ze Ra d i o l o g ic h e ‒ S c u o l a d i Speciali zzazione in Radi ologia\nUniversità Politecnica delle Marche ‒ Ancona (Italy)\nAzienda Ospedali Riuniti ‒ Torrette\nANCONA\nHRTC di un uomo di 80 anni con dispnea e febbre risultato positivo per COVID-19; esame eseguito a 5 giorni dallʼesordio.\n A\nImmagine A: ricostruzione con algoritmo Lung, immagine\nassiale. Si\napprezzano multiple opacità dja “vetro\nsmerigliato“00 cui si associa, in particolare ai lobi polmonari\ninferiori, ispessimento dei setti interlobulari con alispetto a\n"crazy paviſing o" cerchio nero). È anche presente addensamento\nlineare a distribuzione mantellare-subpleurica (freccia nera\npiena).\n B\nImmagine B: Ricostruzione coronale che mostra la distribuzione\nprevalentemente periferica delle opacità a "vetro smerigliato“\n(frecce nere vuote).\n ', 'COVID-19: cas

False

## Upload to Datastore

In [3]:
# https://www.kdnuggets.com/2019/04/text-preprocessing-nlp-machine-learning.html
# Load model
# en_ner_bionlp13cg_md or en_core_sci_lg
#nlp = spacy.load("en_core_sci_lg")

nlp = en_core_sci_lg.load()

In [4]:
# Add pipe features to pipeline 
linker = UmlsEntityLinker(resolve_abbreviations=True)
nlp.add_pipe(linker)

# Add the abbreviation pipe to the spacy pipeline.
abbreviation_pipe = AbbreviationDetector(nlp)
nlp.add_pipe(abbreviation_pipe)

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [5]:
def medicalEntityExtraction(doc):
    # convert text to vector
    display_text = displacy.render(doc,jupyter=True,style='ent')
    annotated_entities = set([(X.text, X.label_) for X in doc.ents])
    return display_text, annotated_entities

In [6]:
def addTask(client, entities_dict):
    key = client.key('case', doc_title)
    task = datastore.Entity(key=key)
    task.update(
        entities_dict)
    client.put(task)
    # Then get by key for this entity
    return client.get(key)

In [15]:
# list of blobs
bucket_name = 'covid19-public-dataset-aketari'
gcs_source_prefix = 'curated_eng_txt'
lst_blobs = storage_client.list_blobs(bucket_or_name=bucket_name, 
                                      prefix=gcs_source_prefix)


for blob in lst_blobs:
    doc_title = blob.name.split('/')[-1].split('.txt')[0]
    
    # download as string
    eng_string = blob.download_as_string().decode('utf-8')
    
    # convert to vector
    doc = nlp(eng_string)
    
    # Extract medical entities
    pattern = 'T(\d{3})'

    UMLS_tuis_entity = {}
    entity_dict = {}


    for idx in range(len(doc.ents)):
        entity = doc.ents[idx]
        entity_dict[entity] = ''
        for umls_ent in entity._.umls_ents:
            entity_dict[entity] = linker.umls.cui_to_entity[umls_ent[0]]

        tui = re.search(pattern, str(entity_dict[entity]))
        if tui:
            UMLS_tuis_entity[str(entity)] = tui.group()
        else:
            UMLS_tuis_entity[str(entity)] = None
            
    # generate dataframes
    entities = list(UMLS_tuis_entity.keys())
    TUIs = list(UMLS_tuis_entity.values())
    df_entities = pd.DataFrame(data={'entity':entities,'TUIs':TUIs})
    df_reference_TUIs = pd.read_csv('UMLS_tuis.csv')
    df_annotated_text_entities = pd.merge(df_entities,df_reference_TUIs,how='inner',on=['TUIs'])
    
    # upload entities to datastore
    entities_dict = {}
    for idx in range(df_annotated_text_entities.shape[0]):
        
        category = df_annotated_text_entities.iloc[idx].values[2]
        
        #TUI = df_annotated_text_entities.iloc[idx].values[1]
        #entities_dict[category].append(TUI)
        
        med_entity = df_annotated_text_entities.iloc[idx].values[0]
        
        
        # Append to list of entities if the key,value pair already exist
        try:
            entities_dict[category].append(med_entity)
            
        except:
            entities_dict[category] = []
            entities_dict[category].append(med_entity)
        
        
        # API call
    key = addTask(datastore_client, entities_dict)
    print('The upload of {} entities is done.'.format(doc_title))

The upload of case1 entities is done.
The upload of case10 entities is done.
The upload of case11 entities is done.
The upload of case12 entities is done.
The upload of case13 entities is done.
The upload of case14 entities is done.
The upload of case15 entities is done.
The upload of case16 entities is done.
The upload of case17 entities is done.
The upload of case18 entities is done.
The upload of case19 entities is done.
The upload of case2 entities is done.
The upload of case20 entities is done.
The upload of case21 entities is done.
The upload of case22 entities is done.
The upload of case23 entities is done.
The upload of case24 entities is done.
The upload of case25 entities is done.
The upload of case26 entities is done.
The upload of case27 entities is done.
The upload of case28 entities is done.
The upload of case29 entities is done.
The upload of case3 entities is done.
The upload of case30 entities is done.
The upload of case31 entities is done.
The upload of case32 entitie

In [8]:
df_annotated_text_entities.head()

,entity,TUIs,Categories
0,HRTC,T059,Laboratory Procedure
1,year,T079,Temporal Concept
2,man,T047,Disease or Syndrome
3,dyspnoea,T047,Disease or Syndrome
4,fever,T109,Organic Chemical


In [21]:
def getCases(datastore_client, filter_dict, limit=10):
    query = datastore_client.query(kind='case')
    
    for key,values in filter_dict.items():
        for value in values:
            query.add_filter(key, '=', value)
    results = list(query.fetch(limit=limit))
    return results

In [25]:
filter_dict = {'Sign or Symptom':['onset symptoms', "chills"]}
getCases(datastore_client,filter_dict)

[<Entity('case', 'case39') {'Quantitative Concept': ['results', 'multiple areas', 'density'], 'Therapeutic or Preventive Procedure': ['dyspnea', 'increased'], 'Disease or Syndrome': ['SaO', 'interstitial pneumonia'], 'Biomedical or Dental Material': ['swab'], 'Gene or Genome': ['PaCO'], 'Mental Process': ['transferred'], 'Cell Component': ['ter ter apia'], 'Entity': ['productive'], 'Body Location or Region': ['Chest CT', 'pharyngeal swab'], 'Spatial Concept': ['externally'], 'Intellectual Product': ['nausea', 'loss appetite', 'altered', 'L', 'CT', 'patent'], 'Population Group': ['woman'], 'Medical Device': ['blood sampling', 'L E erythrocyte sedimentation'], 'Qualitative Concept': ['associated in'], 'Tissue': ['lung parenchyma type frosted'], 'Amino Acid, Peptide, or Protein': ['Lactate Dehydrogenase'], 'Laboratory or Test Result': ['laboratory tests'], 'Idea or Concept': ['patient', 'Patient'], 'Laboratory Procedure': ['normal blood count', 'blood gas analysis'], 'Temporal Concept': [